In [6]:
#!pip install pyproj
#!pip install request

# MANUAL SKETCHBOOK

La finalidad de este software es completar la mayor cantidad de información del estandar Darwin Core a partir de un par de coordenadas, *importante hacer notar que este software hasta el momento no realiza ningun tipo de georreferenciación, solo rellena datos*, esto se realiza a partir de un set de coordenadas, esta pueden ser del campo verbatimCoordinates o decimalLatitude y decimalLongitude, pertenecientes al estandar. Con estos datos será completar los siguientes campos, si es que se encuentra la información necesaria. 

- higherGeography 
- continent 
- country 
- countryCode 
- stateProvince 
- county 
- municipality 
- decimalLongitude, decimalLatitude
- geodeticDatum 
- coordinatePrecision 
- verbatimLatitude, verbatimLongitude
- verbatimCoordinateSystem 

Hasta el momento no se ha podido probar la localización de localidades marinas, pero es un trabajo en progreso. 

Este script se divide en dos clases

- Preparación de la base de datos (db_prep)
- Transformación de coordenadas (coord_transform) 

## Preparación de la base de datos

Esta clase recibe el nombre de "db_prep" y necesita como entrada una base de datos con algunos componentes mínimos del estandar Darwin Core (DwC) para su realización.



## Transformación de coordenadas 

- El campo verbatimCoordintes debe tener las coordenadas en el siguiente formato (dejar en georeferenceRemarks las coordenadas como vienen en su etiqueta original)
        -En el caso de coordenadas en grados minutos y segundos gg°mm'ss" S, gg°mm'ss" W o gg°mm'ss" N, gg°mm'ss" E la importancia radica que debes estar separadas por una coma, misma situación para grados y minutos decimales
        -En el caso de UTM deben ser escritas de la siguiente forma 308617 6272156 19S, la importancia esta que se encuentren separadas por espacio. 
    

In [7]:
#from pyproj import Proj #no se está usando por el momento
import utm
import requests
import json
import pandas as pd
import numpy as np
import math
import re
from GeoParser import *
#from geopy.geocoders import Nominatim #FASTER than api

# TESTING...

In [8]:
import pandas as pd
data=pd.read_csv("new_georref.csv")

In [9]:
%%time
#Testing...
DBPrep(data).verbatim_coordinates_autocomplete()
DBPrep(data).verbatim_coordinate_system_autocomplete()
DBPrep(data).verbatim_latitude_longitude_autocomplete()
DBPrep(data).coordinate_precision_autocomplete()

Wall time: 197 ms


In [10]:
%%time
# Testing ...
#coord_transform(data).utm_to_dg()
CoordTransform(data).dms_to_dg()
CoordTransform(data).geodetic_datum_autocomplete()  
CoordTransform(data).location_autocomplete() 
CoordTransform(data).continent_autocomplete() 
CoordTransform(data).higher_geography_autocomplete() 

Wall time: 2min 6s


In [11]:
data.query("verbatimCoordinates.notna()")

,higherGeography,waterBody,continent,country,countryCode,stateProvince,county,municipality,islandGroup,island,...,verbatimLongitude,verbatimCoordinateSystem,verbatimSRS,georeferencedBy,georeferencedDate,georeferenceProtocol,georeferenceSources,georeferenceVerificationStatus,georeferenceRemarks,localityid
18,South America | Chile | CL | Araucanía | Cautí...,NaN,South America,Chile,CL,Araucanía,Cautín,Vilcun,NaN,NaN,...,"72°2'0.000"" W",degrees minutes seconds,NaN,M. Treumun,2018-09-07,NaN,NaN,NaN,En base a la localidad,DJK/F1K+y0BrVZJC8WqhXXao9psawnd3vgcIBmDwEkM=
21,South America | Chile | CL | Araucanía | Cautí...,NaN,South America,Chile,CL,Araucanía,Cautín,Nueva Imperial,NaN,NaN,...,"72°35'0.000"" W",degrees minutes seconds,NaN,M. Treumun,2018-09-07,NaN,NaN,NaN,En base a la localidad,3y8XajQtDujwAH0aRMXw9TlaiVTjUIFoZ3pKML4qLTY=
28,South America | Chile | CL | Araucanía | Cautí...,NaN,South America,Chile,CL,Araucanía,Cautín,Cunco,NaN,NaN,...,"72°15'0.000"" W",degrees minutes seconds,NaN,M. Treumun,2018-09-07,NaN,NaN,NaN,En base a la localidad,VynSmlTthK9N7jVp8xoEWOQzHgn45zpG2QBAG2dYQC8=
29,South America | Chile | CL | Araucanía | Malle...,NaN,South America,Chile,CL,Araucanía,Malleco,Curacautin,NaN,NaN,...,"71°44'0.000"" W",degrees minutes seconds,NaN,M. Treumun,2018-09-07,NaN,NaN,NaN,En base a la localidad,dj2qw/5FanHksCpVG/Jci5vD3Vsyvld74Dg3xXVb3VQ=
34,South America | Chile | CL | Atacama | Chañara...,NaN,South America,Chile,CL,Atacama,Chañaral,Chanaral,NaN,NaN,...,"70°39'2.488"" W",degrees minutes seconds,NaN,M. Treumun,2018-09-07,NaN,NaN,NaN,En base a la localidad,yDNfsPgjmET2GM8++vd4OK+wJqPKUpvBVsYK/JiLO0k=
67,South America | Chile | CL | Los Lagos | Llanq...,NaN,South America,Chile,CL,Los Lagos,Llanquihue,Puerto Varas,NaN,NaN,...,"72°18'0.000"" W",degrees minutes seconds,NaN,M. Treumun,2018-09-07,NaN,NaN,NaN,En base a la localidad,Z+pWP41/ZGyg01I+zJfdZUWmeWLEjQb5RUmocDhAIcs=
94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Charif Tala,NaN,NaN,Google Earth,NaN,NaN,4eEDc0BC/RD2KMddqR/tZEaBxbe9xL+WbWwLst9Y4Xo=
100,South America | Chile | CL | Valparaíso | ísla...,NaN,South America,Chile,CL,Valparaíso,ísla de Pascua,Ocean Islands,NaN,NaN,...,"80°46'0.000"" W",degrees minutes seconds,NaN,M. Treumun,2018-09-07,NaN,NaN,NaN,En base a la localidad,MyUBTfNzm5z0nwL3PkLxvLqv16cWRzWXXgqY6I4rGs0=
107,South America | Chile | CL | Valparaíso | ísla...,NaN,South America,Chile,CL,Valparaíso,ísla de Pascua,Ocean Islands,NaN,NaN,...,"78°51'0.000"" W",degrees minutes seconds,NaN,M. Treumun,2018-09-07,NaN,NaN,NaN,En base a la localidad,YCgWdFqJbtrW1Yv4keFu6Ld+IpjfOChW5WP7u/oYAoM=


In [12]:
data

,higherGeography,waterBody,continent,country,countryCode,stateProvince,county,municipality,islandGroup,island,...,verbatimLongitude,verbatimCoordinateSystem,verbatimSRS,georeferencedBy,georeferencedDate,georeferenceProtocol,georeferenceSources,georeferenceVerificationStatus,georeferenceRemarks,localityid
0,South America | Chile | CL | Araucanía | Malle...,NaN,South America,Chile,CL,Araucanía,Malleco,Lonquimay,NaN,NaN,...,NaN,NaN,NaN,Charif Tala,NaN,NaN,Google Earth,NaN,NaN,TxmVXS0HuObUcicDzF9JBaWNHK4C/tiPdf9jxpTLMqU=
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WRYd0YGi5ik2bvvJRhC1Ynf4OyfzirN66qbhuD3Fgfc=
2,South-Eastern Pacific Ocean,South-Eastern Pacific Ocean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TtMBF2hQMfzrgMVqyIbTHRP0UF9v4Qn8uNonMtuye/s=
3,South-Eastern Pacific Ocean,South-Eastern Pacific Ocean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gSE0qlewC1j4UoKOU2+A5amKtzc1j/I5h2bhjv2d+Q8=
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tw11N7qFI/XzbJam2Z1HHIoLjzOfoRsDpZEPw5+W4tU=
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,South America | Chile | CL | Región Metropolit...,NaN,South America,Chile,CL,Región Metropolitana de Santiago,Valparaíso,Hijuelas,NaN,NaN,...,NaN,NaN,NaN,Charif Tala,NaN,NaN,Google Earth,NaN,NaN,rbPp7LlP8R78/VBHQiWfqA51+qeeu62rJ8XODXqBm/c=
106,NaN,NaN,South America,Chile,CL,Valparaíso,Valparaíso,Valparaiso,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://www.geonames.org | Google Earth | http...,NaN,"Georreferenciado a través de GeoNames, Geoloca...",HVoHXv8ifqVeUCAYhiDRTuyKmqixjppKV3bwVx7MAbw=
107,South America | Chile | CL | Valparaíso | ísla...,NaN,South America,Chile,CL,Valparaíso,ísla de Pascua,Ocean Islands,NaN,NaN,...,"78°51'0.000"" W",degrees minutes seconds,NaN,M. Treumun,2018-09-07,NaN,NaN,NaN,En base a la localidad,YCgWdFqJbtrW1Yv4keFu6Ld+IpjfOChW5WP7u/oYAoM=
108,South-Eastern Pacific Ocean,South-Eastern Pacific Ocean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/0h1tIuArvSEx1ktY3JjnXCcHGrefNvWljHHsTqfcAs=


In [13]:
driver.quit()